## Google Drive Authentication

In [16]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']


creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)
    service_sheets = build('sheets', 'v4', credentials=creds)
    
    print("** Authentication Complete! **")

except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')


** Authentication Complete! **


## Google Drive API helper functions

In [17]:
import io
from googleapiclient.http import MediaIoBaseDownload
from os import listdir

def getFolderId(service, folderName: str):
    query = "name contains '%s' and mimeType = '%s'" % (folderName, 'application/vnd.google-apps.folder')

    fid = None

    if folderName.startswith('+'):
        return (folderName[1:])

    result = service.files().list(q=query,
                                  pageSize=10, pageToken='',
                                  fields="nextPageToken,files(parents,id,name,mimeType)").execute()
  
    if len(result['files']) == 0:
        print("Folder NOT found")
    else:
        folder = result.get('files')[0]
        fid = folder['id']

    return (fid)

    
def downloadFolder(service, fileId, destinationFolder):
    if not os.path.isdir(destinationFolder):
        os.mkdir(path=destinationFolder)

    results = service.files().list(
        pageSize=300,
        q="parents in '{0}'".format(fileId),
        fields="files(id, name, mimeType)"
        ).execute()

    items = results.get('files', [])

    for item in items:
        itemName = item['name']
        itemId = item['id']
        itemType = item['mimeType']
        filePath = destinationFolder + "/" + itemName

        if itemType == 'application/vnd.google-apps.folder':
            downloadFolder(service, itemId, filePath) # Recursive call
            print("Downloaded folder: {0}".format(itemName))
        elif not itemType.startswith('application/'):
            downloadFile(service, itemId, filePath)
        else:
            print("Unsupported file: {0}".format(itemName))


def downloadFile(service, fileId, filePath):
    # Note: The parent folders in filePath must exist
    request = service.files().get_media(fileId=fileId)
    fh = io.FileIO(filePath, mode='wb')
    
    try:
        downloader = MediaIoBaseDownload(fh, request, chunksize=1024*1024)

        done = False
        while done is False:
            status, done = downloader.next_chunk(num_retries = 2)
    finally:
        fh.close()
        
def deleteFilesInFolder(folder_id):
    results = service.files().list(
        pageSize=300,
        q="parents in '{0}'".format(folder_id),
        fields="files(id, name, mimeType)"
        ).execute()

    items = results.get('files', [])
    for item in items:
        itemId = item['id']
        service.files().delete(fileId=itemId).execute()
        
def uploadFolder(service, folder_id, src_folder):        
    for file in listdir(src_folder):
        print('Uploading: ' + file)
        file_metadata = {'name': file, 'parents': [folder_id]}
        media = MediaFileUpload(os.path.join(src_folder, file), mimetype='image/png')
        file = service.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()

## Download Trait Files!

In [18]:
import shutil

traits_base_filepath = 'Traits'

# Delete previously downloaded trait files
if os.path.isdir(traits_base_filepath):
    shutil.rmtree(traits_base_filepath)

traits_folder_id = getFolderId(service, traits_base_filepath)

downloadFolder(service, traits_folder_id, traits_base_filepath)

print("\n** Download Complete! **")

Downloaded folder: parka
Downloaded folder: background
Downloaded folder: ears
Downloaded folder: blingpiece
Downloaded folder: bling
Downloaded folder: patch
Downloaded folder: satchels
Downloaded folder: quarter zip
Downloaded folder: upper + lower
Downloaded folder: short pants
Downloaded folder: hats + face
Downloaded folder: fur hat
Downloaded folder: ice cream
Downloaded folder: bowl cut
Downloaded folder: money
Downloaded folder: pillow
Downloaded folder: headphones
Downloaded folder: spilled carton
Downloaded folder: bucket hats
Downloaded folder: Flag Patches
Downloaded folder: Patches
Downloaded folder: beanies
Downloaded folder: Patches
Downloaded folder: baseball cap
Downloaded folder: hats
Downloaded folder: walkman
Downloaded folder: halo
Downloaded folder: dots
Downloaded folder: healthbar
Downloaded folder: external
Downloaded folder: beard
Downloaded folder: chinese mask
Downloaded folder: syringe
Downloaded folder: sticky notes
Downloaded folder: glasses
Downloaded fo

## Print all folders, files (BFS)

In [19]:
folder_queue = [traits_base_filepath]

count = 0

#Generate rarity definitions
while folder_queue:
    curr_folder = folder_queue.pop(0)
    
    files = os.listdir(curr_folder)
    files = filter(lambda file: not file.startswith('.'), files)
    
    for file in files:
        print(file)
        if file.lower().endswith('.png'):
            count += 1
        if not file.lower().endswith('.png'):
            folder_queue.append(os.path.join(curr_folder, file))
            
print('\nPNG Count: ', count)

background
short pants
hoodies
quarter zip
face
short sleeves
hats
body
upper + lower
ears
fat pants
slim pants
dress shirt
undergarment upper
hats + face
external
shoes
parka
undergarment lower
background_green.PNG
background_blue.PNG
background_orange.PNG
background_red.PNG
background_yellow.PNG
shorts_black.PNG
shorts_grey.PNG
shorts_red.PNG
shorts_brown.PNG
shorts_blue.PNG
shorts_purple.PNG
shorts_green.PNG
Hoodie_green.PNG
Hoodie_vanilla.PNG
Hoodie_grey.PNG
HoodieLP_grey_yellow.PNG
HoodieLP_green_red.PNG
satchels
HoodieLP_grey_red.PNG
HoodieLP_vanilla_black.PNG
Hoodie_white.PNG
HoodieLP_vanilla_blue.PNG
HoodieLP_vanilla_pink.PNG
HoodieLP_green_grey.PNG
bling
HoodieLP_vanilla_red.PNG
quarterZip_red.PNG
satchels
quarterZip_yellow.PNG
quarterZip_pink.PNG
quarterZip_blue.PNG
patch
bling
quarterZip_green.PNG
quarterZip_grey.PNG
surgical masks
fb
sticky notes
glasses
beard
syringe
chinese mask
hawaiian shirt
Meta Tees
fur hat
baseball cap
bucket hats
beanies
ice cream
headphones
pillow


## Some more helper functions

In [20]:
from collections import deque
import random

# folder: subtrait
# traits_already_picked: keep track
# returns: chosen image(s), new set of already picked traits
def pick_trait_images(folder, traits_already_picked):
    
    png_files, folder_files = get_compatible_files(folder, traits_already_picked)

    if not png_files and not folder_files:
        return None, traits_already_picked
    
    chosen_image, chosen_folders = rarity_chooser(png_files, folder_files)
    
    traits_picked = []
    if chosen_image:
        traits_picked = get_traits_for_image(os.path.join(folder, chosen_image))
    
    # If only PNGs in folder
    if len(folder_files) == 0:
        return [os.path.join(folder, chosen_image)], traits_already_picked + traits_picked
    
    # If only folders in folder
    elif len(png_files) == 0:
        chosen_folder = chosen_folders[0]
        return pick_trait_images(os.path.join(folder, chosen_folder), traits_already_picked)
    
    # Both PNGs and folders in folder
    else:
        chosen_image_path = os.path.join(folder, chosen_image)
        picked_images_for_folders = []
        picked_traits_for_folders = []
        for chosen_folder in chosen_folders:
            picked_image, picked_traits = pick_trait_images(os.path.join(folder, chosen_folder), traits_already_picked + traits_picked)
            picked_images_for_folders.append(picked_image)
            picked_traits_for_folders.append(picked_traits)
            
        return [chosen_image_path] + picked_images_for_folders, traits_already_picked + traits_picked + picked_traits_for_folders
                                                        
# elems: list of PNGs/folders
def rarity_chooser(png_files, folder_files):
    
    if not png_files and not folder_files:
        return None, [None]
    
    elif not png_files and folder_files:
        return None, [weighted_pick(folder_files)]
    
    elif not folder_files and png_files:
        return weighted_pick(png_files), [None]
    
    else:
        chosen_png = weighted_pick(png_files)
        chosen_folders = []
        
        for file in folder_files:
            probability = rarity_dict[file]
            if probability and decision(probability):
                chosen_folders.append(file)
        
        return chosen_png, chosen_folders
    
def weighted_pick(files):
    weights = []
    for file in files:
        if not rarity_dict[file]:
            return random.choice(files)
        weights.append(int(rarity_dict[file]))
    
    choices = random.choices(files, weights=weights)
    return choices[0]
    
def decision(probability):
    return random.random() < probability

# folder:
# traits_already_picked:
# returns: compatible files within folder that are compatible with traits_already_picked
def get_compatible_files(folder, traits_already_picked):
    files = os.listdir(folder)
    
    # Drop pesky ./DSStore files
    files = filter(lambda file: not file.startswith('.'), files)
    
    png_files = []
    folder_files = []
    for file in files:
        if file.lower().endswith('.png'):
            trait_name, color, _ = parse_png_filename(file)            
        
            if is_possible_choice(trait_name, exclusions_dict, traits_already_picked) & \
                is_possible_choice(color, exclusions_dict, traits_already_picked):
                png_files.append(file)
        else:
            if is_possible_choice(file, exclusions_dict, traits_already_picked):
                folder_files.append(file)
   

    return png_files, folder_files

def is_possible_choice(entry, exclusions_dict, traits_already_picked):
    return (entry not in exclusions_dict or not(set(exclusions_dict[entry]) & set(traits_already_picked)))

def get_traits_for_image(chosen_image):
    traits = chosen_image.split('/')
    trait_name, color, _ = parse_png_filename(traits[-1])
    traits[-1] = trait_name
    traits.append(color)
    return traits
    
def parse_png_filename(png_filename):
    png_filename = os.path.splitext(png_filename)[0]
    parsed_attributes = png_filename.split('_')
    if len(parsed_attributes) == 1:
        return parsed_attributes[0], "", ""
    elif len(parsed_attributes) == 2:
        return parsed_attributes[0], parsed_attributes[1], ""
    elif len(parsed_attributes) == 3:
        return parsed_attributes[0], parsed_attributes[1], parsed_attributes[2]
    
def get_last_index(inp_list, elem):
    index = None
    for idx, i in enumerate(inp_list):
        if i == elem:
            index = idx
    return index

## Generate Babies!

In [21]:
from pathlib import Path

# Run this on update of either 1. layering order and 2. exclusions sheet

sheet = service_sheets.spreadsheets()

LAYERING_ORDER_SPREADSHEET_ID = '1aHC5g3mPSJGFAPF7UiQXDnV9BBUJCwnx8QHkMMip6uI'
LAYERING_ORDER_RANGE = 'A1:A'

EXCLUSIONS_SPREADSHEET_ID = '1S3Gbg24gwCmn_2AwThAIlRN9F0PEbA2v1Kn3QkaTjeY'
EXCLUSIONS_RANGE = 'A1:B'

RARITY_DEV_SPREADSHEET_ID = '1NhV9RmhDjme4MA4QMzMOTn9tdDoK-OL6kWK0XSG_G8M'
RARITY_SPREADSHEET_ID = '1rvtwtSps-1g35rhXMkt8zU1A7PV62FmFgwPDvZuTLL4'

RARITY_RANGE = 'A1:B'


result = sheet.values().get(spreadsheetId=LAYERING_ORDER_SPREADSHEET_ID, range=LAYERING_ORDER_RANGE).execute()
values = result.get('values', [])

ordered_traits = [item for sublist in values for item in sublist]

print("** Traits Ordering ingested! **\n")
    
## Generate warnings for trait exclusions
traits_dfs_order = []
folder_queue = ordered_traits[::-1]
folder_queue = ['Traits/' + path for path in folder_queue]

while folder_queue:
    curr_folder = folder_queue.pop(-1)
    traits_dfs_order.append(Path(curr_folder).stem)
    
    files = os.listdir(curr_folder)
    files = filter(lambda file: not file.startswith('.'), files)
    
    for file in files:
        if file.lower().endswith('.png'):
            trait, color, secondary_color = parse_png_filename(file)
            traits_dfs_order.append(trait)
            if color != "":
                traits_dfs_order.append(color)
            if secondary_color != "":
                traits_dfs_order.append(secondary_color)
        else:
             folder_queue.append(os.path.join(curr_folder, file))
            

result = sheet.values().get(spreadsheetId=EXCLUSIONS_SPREADSHEET_ID, range=EXCLUSIONS_RANGE).execute()
values = result.get('values', [])

# Warning generation
for val in values:
    if val[0] not in traits_dfs_order:
        print("\n** WARNING -- ", val[0], "in exclusions not a valid entry")
        break
    exclusions = val[1].split(',')
    for e in exclusions:
        if not get_last_index(traits_dfs_order, e):
            print("\n** WARNING -- ", e, "in exclusions is not a valid entry")
        else:
            if not get_last_index(traits_dfs_order, e) > traits_dfs_order.index(val[0]) and e != val[0]:
                print("\n** WARNING -- Ordering of ", val[0], ",", e, "in exclusions is not valid")

exclusions_dict = {}
for val in values:
    exclusions = val[1].split(',')
    for e in exclusions:
        if e not in exclusions_dict:
                exclusions_dict[e] = []
        exclusions_dict[e].append(val[0])

print("\n** Traits Exclusions ingested! **\n")    

# Rarity ingestion
result = sheet.values().get(spreadsheetId=RARITY_DEV_SPREADSHEET_ID, range=RARITY_RANGE).execute()
values = result.get('values', [])

rarity_dict = {}
for val in values:
    if len(val) == 1:
        rarity_dict[val[0]] = None
    elif len(val) == 2:
        rarity_dict[val[0]] = val[1]
    

print("\n Rarities Ingested!")

        
images_count = 50

num_traits_excluding_body = 5

babies_base_filepath = 'Babies'

** Traits Ordering ingested! **


** Traits Exclusions ingested! **


 Rarities Ingested!


In [25]:
from PIL import Image, ImageDraw, ImageChops

# top level
def generate_baby(ordered_traits, traits_base_filepath, babies_base_filepath):
    # BFS for trait images selection
    picked_trait_images = []
    picked_traits = []
    base_traits = ordered_traits
    
    # 1. choose the body and background trait
    background_trait_folder = base_traits.pop(0)
    curr_trait_dir = os.path.join(traits_base_filepath, background_trait_folder)
    picked_trait_image, picked_traits = pick_trait_images(curr_trait_dir, picked_traits)
    picked_trait_images.extend(picked_trait_image)
    
    body_trait_folder = base_traits.pop(0)
    curr_trait_dir = os.path.join(traits_base_filepath, body_trait_folder)
    picked_trait_image, picked_traits = pick_trait_images(curr_trait_dir, picked_traits)
    picked_trait_images.extend(picked_trait_image)
    
    # 2. Choose 0-2 face traits
    face_traits = ['face', 'hats', 'hats + face', 'ears']
    num_face_traits = random.randint(0, 2)
    chosen_face_traits = [face_traits[i] for i in sorted(random.sample(range(len(face_traits)), num_face_traits))]
    
    
    # 3. Randomly pick `num_traits_excluding_body` traits
    base_traits = list(set(base_traits) - set(face_traits))
    num_traits_excluding_body_face = num_traits_excluding_body - num_face_traits
    chosen_base_traits = [base_traits[i] for i in sorted(random.sample(range(len(base_traits)), num_traits_excluding_body_face))]
    
    unsorted_chosen_traits = chosen_base_traits + chosen_face_traits 
    final_chosen_traits = [trait for x in ordered_traits for trait in unsorted_chosen_traits if trait == x]

    print("Chosen traits: " + str(final_chosen_traits))
    
    while final_chosen_traits:
        curr = final_chosen_traits.pop(0)
        if curr in exclusions_dict and (set(exclusions_dict[curr]) & set(picked_traits)):
            continue
            
        curr_trait_dir = os.path.join(traits_base_filepath, curr)
        
        # pick based on exclusions
        picked_trait_image, picked_traits = pick_trait_images(curr_trait_dir, picked_traits)
        if picked_trait_image: 
            picked_trait_images.extend(picked_trait_image)
        
        
    # Layer the images 
    x, y = Image.open(picked_trait_images[0]).size
    final_baby_image = Image.new('RGB', (x, y), (228, 150, 150))
    
    for trait_image in picked_trait_images:
        chosen_image = Image.open(trait_image)
        chosen_image = ImageChops.offset(chosen_image, 90, 0)
        final_baby_image.paste(chosen_image, (0, 0), chosen_image)
    
    # Crop to increase baby appearance
    final_baby_image = final_baby_image.crop((0, 140, final_baby_image.width - 140, final_baby_image.height))
    
    return final_baby_image


## Generate babies!

# Delete previously created baby files
if os.path.isdir(babies_base_filepath):
    shutil.rmtree(babies_base_filepath)

os.mkdir(babies_base_filepath)

for i in range(images_count):
    final_baby_image = generate_baby(ordered_traits.copy(), traits_base_filepath, babies_base_filepath)
    
    # Write the image to file
    final_baby_image_file = '{:d}_lonely_baby.PNG'.format(i)
    final_baby_image.save(os.path.join(babies_base_filepath, final_baby_image_file))
    
    print("Completed " + final_baby_image_file)

print("\n**Image generation complete! **")

Chosen traits: ['short pants', 'slim pants', 'undergarment upper', 'fat pants', 'hoodies']
Completed 0_lonely_baby.PNG
Chosen traits: ['slim pants', 'fat pants', 'quarter zip', 'hoodies', 'external']
Completed 1_lonely_baby.PNG
Chosen traits: ['undergarment lower', 'slim pants', 'undergarment upper', 'dress shirt', 'hats']
Completed 2_lonely_baby.PNG
Chosen traits: ['undergarment lower', 'fat pants', 'short sleeves', 'face', 'hats + face']
Completed 3_lonely_baby.PNG
Chosen traits: ['short pants', 'undergarment upper', 'fat pants', 'hats', 'hats + face']
Completed 4_lonely_baby.PNG
Chosen traits: ['undergarment upper', 'fat pants', 'short sleeves', 'dress shirt', 'quarter zip']
Completed 5_lonely_baby.PNG
Chosen traits: ['undergarment lower', 'slim pants', 'short sleeves', 'face', 'hats']
Completed 6_lonely_baby.PNG
Chosen traits: ['undergarment lower', 'short sleeves', 'dress shirt', 'quarter zip', 'hoodies']
Completed 7_lonely_baby.PNG
Chosen traits: ['undergarment upper', 'fat pants

## Upload Babies to Google Drive

In [23]:
#babies_folder_id = getFolderId(service, babies_base_filepath)
#deleteFilesInFolder(babies_folder_id)
#uploadFolder(service, babies_folder_id, os.path.join(babies_base_filepath))